#Habitat Suitability under Climate Change

## Study Overview

Understanding grasslands is essential in understanding global carbon sinks. In particular, understanding the layer of soil that lies directly beneath the surface. The presence of Andropogon gerardii, or Big Bluestem, in grassland ecosystems has been significantly connected to the quantity of carbon stored in grassland ecosystems. Where there is more bluestem, there is more stored carbon. How has this changed in recent history - have there been any changes in Bluestem's range? How might Bluestem's range change in the future, given various climate scenarios? 

In this study, I will examine Bluestem's historical and future range across the Buffalo Gap and Little Missouri national grasslands. I will use several different variables to come up with a suitability raster, which will show areas in which the habitat is suitable for bluestem. I will look at elevation ranges as well as soil variables, and I will combine this with climate projection scenarios to determine ways in which the bluestem's habitat range might change.


### Pseudocode outline (by notebooks):

- hab_01_gbif: This notebook accesses the species occurrence information for bluestem from the GBIF database.
- hab_02_grassland_boundaries: Accesses the grassland boundary polygons from the USFS website. It then selects the specific grasslands for the study using the NationalGR number, and prints the boundaries on top of ESRI map tiles.
- hab_03a_soil_urls: Compiles a list of urls from the Polaris soil characteristics database. The code selects soil tiles based on the latitude and longitude boundaries of the grassland polygons. Currently this notebook is looking at soil pH values.
- hab_03b_soil_urls2: This notebook is intended to look at a second set of soil characteristics (soil type).
- hab_04_soil_tiles: Prints out the Polaris soil tiles.
- hab_05a_elevation1: Prints out an SRTM Digital Elevation Model for the Sheyenne national grassland.
- hab_05b_elevation2: Prints out an SRTM Digital Elevation Model for the Curlew national grassland.
- hab_06_macav2: Runs the maca V2 climate projection scenario.
- hab_07a_fuzzy_logic: Contains the fuzzy logic model to determine habitat suitability.
- hab_07b_fuzzy_logic2: Contains the fuzzy logic model to determine habitat suitability.
- hab_08_suitability: Uses raster multiplication to calculate a suitability score for bluestem.


# Species Overview: Andropogon gerardii (Big Bluestem)

In terms of global carbon storage and sequestration, soil health is critical. Globally, soil organic carbon holds about 1550 Pg of carbon. Soil inorganic carbon holds about 950 Pg, which together make soil the third largest carbon pool on Earth after the geologic pool (4130 Pg, including fossil fuels) and the oceanic pool (38000 Pg)(Source 1). This figure represents around 80% of the carbon found in terrestrial ecosystems (Source 2).

In one study, the species Andropogon gerardii, commonly called Big Bluestem, was found to have a positive association with carbon storage, and was used as an indicator species (Source 3).

Andropogon gerardii habitat characteristics and tolerances:
- Soil type: Lowland silt and clay loam
- Soil horizon > 10in
- elevation: 
        CO     3,400 to 9,500 feet (1,036-2,896 m)
        NM     3,500 to 9,000 feet (1,067-2,743 m)
        MT     3,300 to 4,000 feet (1,006-1,219 m)
        WY     3,600 to 9,000 feet (1,067-2,743 m)
(Source 4)
- Optimal soil temperature: 25C. "Soil temperatures below 20 °C caused significant reductions in foliar nitrogen (N) and phosphorus(P) concentration". "reduced photosynthetic rates observed at 5 and 10 °C Tsoil" "Net photosynthesis was < 12 /^mol m"^ s"' at 5 and 10 °C Tsoil and > 20 micromol\ m^-2 s^-1 at 15-40 °C.". Based on the graphs in this study, it seems that bluestem would be growth-limited above 30 and below 20 degrees C, and severely limited outside 15-35 degrees. (Source 5)
- min soil temp -15C (source 8)
- well drained or moist soil, full sun (Source 8)
- Precip: 10-60mm / yr (source7)
- pH range: 6.0-7.5 (source 10). Another source (11) says that it does fine with pH 5.5, so we'll use this as out lower-range tolerance. Optimal then is 6.5 (halfway)

Sources:
1) https://royalsocietypublishing.org/doi/pdf/10.1098/rstb.2007.2185
2) https://www.nature.com/scitable/knowledge/library/soil-carbon-storage-84223790/#:~:text=The%20amount%20of%20C%20in,in%20soil%20(Lal%202008).
3) https://pmc.ncbi.nlm.nih.gov/articles/PMC8367897/
4) https://www.fs.usda.gov/database/feis/plants/graminoid/andger/all.html
5) https://nph.onlinelibrary.wiley.com/doi/epdf/10.1111/j.1469-8137.1992.tb01804.x
6) https://plants.usda.gov/DocumentLibrary/factsheet/pdf/fs_ange.pdf
7) https://plants.usda.gov/DocumentLibrary/plantguide/pdf/pg_scsc.pdf
8) https://pfaf.org/user/Plant.aspx?LatinName=Andropogon+gerardii 
9) https://link.springer.com/article/10.1023/A:1020320214750 
10) https://greatbasinseeds.com/product/big-bluestem/?srsltid=AfmBOoqeQLl_4uKA8wwv_4x4rFgQ7XERV07BlzAzxuMDXfQcyyM8s26m
11) https://halehabitat.com/products/big-bluestem

In [ ]:
pip install pygbif

In [1]:
def import_packages():

    ### load packages

    import os
    from glob import glob
    import pathlib

    ### reproducible file paths

    ### gbif packages
    import pygbif.occurrences as occ
    import pygbif.species as species
    from getpass import getpass

    ### packages for unzipping and handling gbif data
    import zipfile
    import time

    ### deal with spatial data
    import geopandas as gpd
    import xrspatial

    ### deal with other types of data
    import numpy as np
    import pandas as pd
    import rioxarray as rxr
    import rioxarray.merge as rxrm

    ### indalid geometries
    from shapely.geometry import MultiPolygon, Polygon

    ### visualizing
    import holoviews as hv
    import hvplot.pandas
    import hvplot.xarray
import_packages

<function __main__.import_packages()>

In [2]:
def file_paths():

    ### Make reproducible file paths
    data_dir = os.path.join(
        ### home directory
        pathlib.Path.home(),
        
        ### eda directory
        'earth-analytics',
        'data',

        ### Project dir
        'hab_suit'
    )

    ### make the dir
    os.makedirs(data_dir, exist_ok=True)
file_paths

<function __main__.file_paths()>

Note: GBIF login is not working, however I kept the code in case it can be fixed!

In [3]:
### set gbif dir
gbif_dir = os.path.join(data_dir, 'gbif_ponderosa')


### access gbif
reset_credentials=True

### enter gbif username, password, email
credentials=dict(
	GBIF_USER=(input, 'GBIF username:'),
	GBIF_PWD=(getpass, 'GBIF password'),
	GBIF_EMAIL=(input, 'GBIF email')
)
for env_variable, (prompt_func, prompt_text) in credentials.items():

	### delete credential from the environment
	if reset_credentials and (env_variable in os.environ):
		os.environ.pop(env_variable)

	### ask for credential and save to environment
	if not env_variable in os.environ:
		os.environ[env_variable] = prompt_func(prompt_text)



NameError: name 'os' is not defined

In [4]:
### species names
species_name = 'Andropogon gerardii'

### species info for gbif
species_info = species.name_lookup(species_name,
					rank = 'SPECIES')

### grab the first resule
first_result = species_info['results'][0]

### get species key
species_key = first_result['nubKey']

### check on that
first_result['species'], species_key



NameError: name 'species' is not defined

In [5]:
### assign species code
species_key = 4932035

In [6]:
### set a file pattern
gbif_pattern = os.path.join(gbif_dir,
				'*.csv')

NameError: name 'os' is not defined

###Species Description:

In terms of global carbon storage and sequestration, soil health is critical. Globally, soil organic carbon holds about 1550 Pg of carbon. Soil inorganic carbon holds about 950 Pg, which together make soil the third largest carbon pool on Earth after the geologic pool (4130 Pg, including fossil fuels) and the oceanic pool (38000 Pg)(Source 1). This figure represents around 80% of the carbon found in terrestrial ecosystems (Source 2).

In one study, the species Andropogon gerardii, commonly called Big Bluestem, was found to have a positive association with carbon storage, and was used as an indicator species (Source 3).

Andropogon gerardii habitat characteristics and tolerances:
- Soil type: Lowland silt and clay loam
- Soil horizon > 10in
- elevation: 
        CO     3,400 to 9,500 feet (1,036-2,896 m)
        NM     3,500 to 9,000 feet (1,067-2,743 m)
        MT     3,300 to 4,000 feet (1,006-1,219 m)
        WY     3,600 to 9,000 feet (1,067-2,743 m)
(Source 4)
- Optimal soil temperature: 25C. "Soil temperatures below 20 °C caused significant reductions in foliar nitrogen (N) and phosphorus(P) concentration". "reduced photosynthetic rates observed at 5 and 10 °C Tsoil" "Net photosynthesis was < 12 /^mol m"^ s"' at 5 and 10 °C Tsoil and > 20 micromol\ m^-2 s^-1 at 15-40 °C.". Based on the graphs in this study, it seems that bluestem would be growth-limited above 30 and below 20 degrees C, and severely limited outside 15-35 degrees. (Source 5)
- min soil temp -15C (source 8)
- well drained or moist soil, full sun (Source 8)
- Precip: 10-60mm / yr (source7)
- pH 6.0-7.5 (source 10)

Sources:
1) https://royalsocietypublishing.org/doi/pdf/10.1098/rstb.2007.2185
2) https://www.nature.com/scitable/knowledge/library/soil-carbon-storage-84223790/#:~:text=The%20amount%20of%20C%20in,in%20soil%20(Lal%202008).
3) https://pmc.ncbi.nlm.nih.gov/articles/PMC8367897/
4) https://www.fs.usda.gov/database/feis/plants/graminoid/andger/all.html
5) https://nph.onlinelibrary.wiley.com/doi/epdf/10.1111/j.1469-8137.1992.tb01804.x
6) https://plants.usda.gov/DocumentLibrary/factsheet/pdf/fs_ange.pdf
7) https://plants.usda.gov/DocumentLibrary/plantguide/pdf/pg_scsc.pdf
8) https://pfaf.org/user/Plant.aspx?LatinName=Andropogon+gerardii 
9) https://link.springer.com/article/10.1023/A:1020320214750 
10) https://greatbasinseeds.com/product/big-bluestem/?srsltid=AfmBOoqeQLl_4uKA8wwv_4x4rFgQ7XERV07BlzAzxuMDXfQcyyM8s26m